In [1]:
# ## inputs
water_xyz = water = """3

H\t0.7493682\t0.0\t0.4424329
O\t0.0\t0.0\t-0.1653507
H\t-0.7493682\t0.0\t0.4424329
#     """
# geometry = water_xyz[3:]

methane = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

geometry = methane

In [2]:
from nbed.embed import nbed_driver


n_active_atoms=2
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
convergence = 1e-10
charge = 0
mu_level_shift=1e6
run_ccsd = True
run_fci = True
max_ram = 8_000


loc_method = "ibo" #"spade" #"pipekmezey" "ibo" "boys"
run_mu_shift = True
run_huzinaga = True


exp_obj = nbed_driver(
    geometry ,
    n_active_atoms,
    basis,
    xc_functional,
    convergence,
    localization_method= loc_method,
    run_mu_shift = run_mu_shift,
    run_huzinaga = run_huzinaga,
    mu_level_shift = mu_level_shift,
    run_ccsd_emb = run_ccsd,
    max_ram_memory = max_ram,
    run_fci_emb=run_fci,
    run_global_fci=True)


/home/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [3]:
exp_obj.run_exp()

 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 5.56e-10
embedded HF energy MU_SHIFT: -16.08934263399464, converged: True
CCSD Energy MU shift:
	%s -39.65265921699233
FCI Energy MU shift:
	%s -39.85904849137096
core constant: 0
embedded HF energy HUZINAGA: -16.089342573105355, converged: True
CCSD Energy HUZ:
	%s -39.65265916285188
FCI Energy HUZ:
	%s -39.85904839550709
core constant: 0
num e emb: 4
[0 2]
[1 3 4]


In [4]:
from nbed.embed import get_qubit_hamiltonian

qubit_H_mu = get_qubit_hamiltonian(exp_obj.molecular_ham_MU, 'jordan_wigner')
qubit_H_huz = get_qubit_hamiltonian(exp_obj.molecular_ham_HUZ, 'jordan_wigner')

In [5]:
mu_classical_E =  exp_obj.classical_energy_MU
huz_classical_E =  exp_obj.classical_energy_HUZ

In [6]:
from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [7]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [8]:
import numpy as np
mu_full_quantum = eig_vals_emb_mu[0] + mu_classical_E
huz_full_quantum = eig_vals_emb_huz[0] + huz_classical_E

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

-39.85904849145011
-39.85904839550722
9.594288741254786e-08


In [9]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

4